# Оглавление:

1. [Финальная очистка данных по метапризнакам и перевод в один большой csv.](#Очистка-данных-по-метапризнакам.)
2. [Препроцессинг содержимого.](#Препроцессинг-выборки)
3. [Подготовка данных для обучения embedding'ов.](#Препроцессинг-входных-данных)
4. [Осмотр существующих embedding'ов для русского языка.](#Получение-embedding'ов)
5. [Обучение lstm сетки со своими embedding'ами.](#Загружаем-данные)
6. [Обучение лог.регрессии.](#bag-of-words)

In [15]:
%pylab inline
import numpy as np
import seaborn as sns
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.layers import Bidirectional, Multiply, Conv1D, MaxPooling1D, Concatenate, BatchNormalization
#from extra_layers import AttentionWithContext 
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import  Embedding, LSTM, Input, Multiply, Dot, Dense, CuDNNLSTM,CuDNNGRU, Dropout, TimeDistributed


Populating the interactive namespace from numpy and matplotlib


/home/grigory/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['seed']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
seed = 7
np.random.seed(seed)
params = dict(
    seed = seed,  
)

## Готовим данные

In [3]:
df = pd.read_csv('data/dataset.csv')
df

,author,text
0,Владимир Евгеньевич Жаботинский,\n\n\n\nУважаемый г-н Н. Н. В своем письме Вы ...
1,Михаил Юрьевич Лермонтов,"\n\n\n\nЯ хочу рассказать вам историю женщины,..."
2,Антон Павлович Чехов,"\n\n\n\nВыдержав экзамен, Гвоздиков сел на кон..."
3,Михаил Юрьевич Лермонтов,\n\n\n\n\n\nРедеют бледные туманы Над бездной ...
4,Денис Васильевич Давыдов,\n\n\n\n\n\nТолстой[1] молчит! — неужто пьян? ...
5,Николай Михайлович Языков,\n\n\n\n\n\nNunc est bibendum: nunc pede liber...
6,Денис Васильевич Давыдов,\n\n\n\n\n\nТолстой[1] молчит! — неужто пьян? ...
7,Антон Павлович Чехов,"\n\n\n\nНочью, часов в 12, по Тверскому бульва..."
8,Александр Сергеевич Пушкин,"\n\n\n\n\n\nБог помочь вам, друзья мои, В забо..."
9,Антон Антонович Дельвиг,"\n\n\n\n\n\n\n\nВ третий раз, мои друзья, Вам ..."


In [10]:
from word_lstm_preprocess import simple_tokenizer, filter_chars
tokenized = filter_chars(df['text']).apply(simple_tokenizer)

,author,texts
0,3,"[asd, mde, dfed]"
1,3,"[ke, mde]"


In [37]:
tiny_df = df[:1000]

inputlen = 2
def split_text(text, classlabel):
    texts = np.array_split(text, int(text.shape[0]/inputlen))
    return pd.DataFrame({
        'text': texts, 
        'author': classlabel
    })
def harmonize_textsdf(df, inputlen):
    
    to_remove = []
    for i, sample in df.iterrows():
        sample, label = sample['text'], sample['author']
        if len(sample) > 1.5*inputlen:
            to_remove.append(i)
            df = pd.concat([df, split_text(sample, label)], axis=0)
    remain = set(df.index) - set(to_remove)
    df = df.loc[list(remain)]
    return df

harmonize_textsdf(tiny_df, 2500)

AttributeError: 'str' object has no attribute 'shape'

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# encode classes
le = LabelEncoder()
le.fit(labels)
y = le.transform(labels)

y = keras.utils.to_categorical(y)
# do train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.3, stratify=y_train)

Datasplit ratio: Train:0.640, Val:0.160, Test:0.200
CPU times: user 20.6 s, sys: 84 ms, total: 20.7 s
Wall time: 20.7 s


### Загружаем embedding'и

In [ ]:
from utils import DriveWrapper, load_obj
gdw = DriveWrapper()
gdw.dowload('my_plus_corpora_emb')
emb = load_obj('my_plus_corpora_emb')

### Готовим модель

In [56]:
emb_dim = 300
hidden_size = 300

def get_encoder(input_length, emb_weights=None):
    encoder = Sequential()
    encoder.add(Embedding(output_dim=emb_dim,
                              input_dim=MAX_NB_WORDS,
                              input_length=input_length,
                              weights=[emb_weights], 
                              mask_zero=not gpu,
                              trainable=False))
    encoder.add(Dropout(0.15))    
    encoder.add(Bidirectional(CuDNNLSTM(units=hidden_size, return_sequences=True)))
    #encoder.add(Dropout(0.25))
    #encoder.add(BatchNormalization())
    #ncoder.add(CuDNNGRU(units=hidden_size, return_sequences=True)) 
    encoder.add(AttentionWithContext())

    return encoder

def get_classifer(emb, n_classes=64):
    inp = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    encoded = get_encoder(MAX_SEQUENCE_LENGTH, 
                          emb_weights=emb)(inp)
    out = Dense(n_classes, activation = "softmax") (encoded)
    return  Model(response_input, out)
    

# model = Sequential()
# model.add(Bidirectional(LSTM(50)))#, return_sequences=True)))
# #model.add(GlobalMaxPool1D())
# #model.add(Dropout(0.1))
# #model.add(Dense(50, activation="relu"))
# model.add(Dropout(0.1))


In [57]:
optimizer = optimizers.Adam(lr=1e-2)#, decay=5e-3),
model = get_classifier(emb)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,  metrics=['accuracy'])

In [ ]:
callbacks = [ #EarlyStopping(monitor='val_acc', patience=15, verbose=1), 
             ModelCheckpoint('lstm.h5', monitor='val_acc',verbose=1, save_best_only=True),
             ReduceLROnPlateau(patience=3)
]

In [58]:
n_epochs = 25
batch_size = 125
model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, 
          validation_data=(X_val,y_val), 
          callbacks=, 
          verbose=1);

Train on 7060 samples, validate on 1766 samples
Epoch 1/60
7060/7060 [==============================] - 136s 19ms/step - loss: 3.6691 - acc: 0.1278 - val_loss: 3.5172 - val_acc: 0.1569
Epoch 2/60
7060/7060 [==============================] - 145s 21ms/step - loss: 3.6383 - acc: 0.1390 - val_loss: 3.5712 - val_acc: 0.1410
Epoch 3/60
7060/7060 [==============================] - 140s 20ms/step - loss: 3.6603 - acc: 0.1330 - val_loss: 3.5783 - val_acc: 0.1059
Epoch 4/60
7060/7060 [==============================] - 193s 27ms/step - loss: 3.6544 - acc: 0.1258 - val_loss: 3.5771 - val_acc: 0.1382
Epoch 5/60
7060/7060 [==============================] - 190s 27ms/step - loss: 3.6382 - acc: 0.1360 - val_loss: 3.5781 - val_acc: 0.1410
Epoch 6/60
7060/7060 [==============================] - 196s 28ms/step - loss: 3.6200 - acc: 0.1370 - val_loss: 3.5454 - val_acc: 0.1467
Epoch 7/60
3072/7060 [============>.................] - ETA: 1:40 - loss: 3.5910 - acc: 0.1471

KeyboardInterrupt: 

In [39]:
# Final evaluation of the model
tr_scores = model.evaluate(X_train, y_train, verbose=1)
print("Train Accuracy: %.2f%%" % (tr_scores[1]*100))
val_scores = model.evaluate(X_val, y_val, verbose=1)
print("Val. Accuracy: %.2f%%" % (val_scores[1]*100))
te_scores = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy: %.2f%%" % (te_scores[1]*100))

7060/7060 [==============================] - 37s 5ms/step

- Char LSTM  - [paper1](https://papers.nips.cc/paper/5782-character-level-convolutional-networks-for-text-classification.pdf)
- LSTM и CNN  https://github.com/alexeyev/CNN-for-Sentence-Classification-in-Keras